In [2]:
import pickle
import os
import time
import numpy as np
import os 
import shutil

import pyximport; pyximport.install()
from LEF_Dynamics import LEFTranslocatorDirectional

import warnings
import h5py 
import glob

## Defining simulation parameters

In [3]:

# -------defining parameters----------
#  -- basic loop extrusion parameters
SEPARATION = 200
LIFETIME = 300
N = 10000   # number of monomers
smcStepsPerBlock = 1  # now doing 1 SMC step per block 

trajectoryLength=100000


birthArray = np.zeros(N, dtype=np.double) + 0.1
deathArray = np.zeros(N, dtype=np.double) + 1. / LIFETIME
stallDeathArray = np.zeros(N, dtype=np.double) + 1 / LIFETIME
pauseArray = np.zeros(N, dtype=np.double)

stallList = [1000, 3000, 5000, 7000, 9000]
stallLeftArray = np.zeros(N, dtype = np.double)
stallRightARray = np.zeros(N, dtype = np.double)
for i in stallList:
    stallLeftArray[i] = 0.8
    stallRightARray[i] = 0.8

LEFNum = N // SEPARATION
LEFTran = LEFTranslocatorDirectional(birthArray, deathArray, stallLeftArray, stallRightARray, pauseArray,
                                     stallDeathArray, LEFNum)
folder = "trajectory"
if not os.path.exists(folder):
    os.mkdir(folder)
    
with h5py.File("trajectory/LEFPositions.h5", mode='w') as myfile:

    dset = myfile.create_dataset("positions", 
                                 shape=(trajectoryLength, LEFNum, 2), 
                                 dtype=np.int32, 
                                 compression="gzip")
    steps = 50 
    bins = np.linspace(0, trajectoryLength, steps, dtype=int)
    for st,end in zip(bins[:-1], bins[1:]):
        cur = []
        for i in range(st, end):
            LEFTran.steps(1)
            LEFs = LEFTran.getLEFs()
            cur.append(np.array(LEFs).T)
        cur = np.array(cur)
        dset[st:end] = cur
    myfile.attrs["N"] = N
    myfile.attrs["LEFNum"] = LEFNum




In [5]:
with h5py.File("trajectory/LEFPositions.h5", mode='r') as myfile:

    dset = myfile["positions"][:]
dset

array([[[ 242,  244],
        [3896, 3898],
        [4834, 4836],
        ...,
        [ 886,  888],
        [3852, 3855],
        [4511, 4513]],

       [[ 241,  245],
        [3895, 3899],
        [4833, 4837],
        ...,
        [ 885,  889],
        [3851, 3856],
        [4510, 4514]],

       [[ 240,  246],
        [3894, 3900],
        [4832, 4838],
        ...,
        [ 884,  890],
        [3850, 3857],
        [4509, 4515]],

       ...,

       [[5557, 5666],
        [8102, 8314],
        [1001, 1274],
        ...,
        [1408, 1761],
        [2087, 2254],
        [4594, 4797]],

       [[5556, 5667],
        [8102, 8314],
        [1001, 1275],
        ...,
        [1408, 1761],
        [2087, 2254],
        [4594, 4797]],

       [[5555, 5668],
        [8102, 8314],
        [1001, 1276],
        ...,
        [1408, 1761],
        [2087, 2254],
        [4594, 4797]]], dtype=int32)